# CmdCaliper Lab 練習
## 注意事項
1. 請先註冊 Huggingface 帳號至 [CyCraftAI/CmdCaliper-small](https://huggingface.co/CyCraftAI/CmdCaliper-small) 填寫分享使用資料同意書
2. 環境使用 Colab 進行，務必先擁有一個 Google 帳號並開啟 [Colab](https://colab.google/)
3. 如果有使用到 CmdCaliper 的 [synthesize_cmds.py](synthesize_cmds.py) 或 [synthesize_positive_cmds.py](synthesize_positve_cmds.py) 時，會需要 OpenAI 的 API_KEY，這可能會產生費用
4. 關於 OpenAI 的更多資訊請參閱 [OpenAI Platform](https://platform.openai.com/docs/overview)
## Lab 說明
此次練習包含 Lab0、Lab1、Lab2 三項，詳細說明請參閱下方題目說明處。
1. 熟悉 Colab 及試用 CmdCaliper
2. 根據敘述找到最接近的 cmdline
3. 根據給定的 cmdline，推論出三個可能的下一步指令

---

## Lab 0 - 熟悉 Colab 及試用 CmdCaliper

In [14]:
print("Hello world")

Hello world


### Bash 操作
Google Colab 最強大的功能是能夠免費使用雲端 GPU。您可以通過在指令前添加 ! 來執行大多數的 Bash 指令。
* 從 Runtime -> Change Runtime Type -> Hardware Acceleration 開啟 GPU。
* 整個 Colab 在雲端虛擬機器 (VM) 中運行。
* 目前的 Colab 筆記本是運行在 Ubuntu 上。

In [15]:
!ls -l
!pwd

total 152
-rw-rw-r-- 1 is1ab is1ab   438 十二  6 10:30 credential_config_template.yaml
-rw-rw-r-- 1 is1ab is1ab   289 十二  6 10:52 credential_config.yaml
drwxrwxr-x 2 is1ab is1ab  4096 十二  6 13:58 dataset
-rw-rw-r-- 1 is1ab is1ab  3732 十二  6 10:30 evaluate.py
-rw-rw-r-- 1 is1ab is1ab  2486 十二 12 13:51 lab1_cmd_retrieval.py
-rw-rw-r-- 1 is1ab is1ab  2926 十二  6 15:45 lab1_desc_to_cmd.py
-rw-rw-r-- 1 is1ab is1ab 85610 十二 12 14:20 lab.ipynb
drwxrwxrwx 2 is1ab is1ab  4096 十二  6 11:07 output
-rw-rw-r-- 1 is1ab is1ab  5777 十二  6 10:30 README.md
-rw-rw-r-- 1 is1ab is1ab   150 十二  6 10:55 requirements.txt
drwxrwxr-x 3 is1ab is1ab  4096 十二  6 10:51 src
-rw-rw-r-- 1 is1ab is1ab  2926 十二  6 11:29 synthesize_cmds.py
-rw-rw-r-- 1 is1ab is1ab  3223 十二  6 10:30 synthesize_positive_cmds.py
-rw-rw-r-- 1 is1ab is1ab  6470 十二  6 14:04 test.ipynb
-rw-rw-r-- 1 is1ab is1ab  4597 十二  6 15:45 test.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/is1ab/code2/CmdCaliper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04 LTS"
NAME="Ubuntu"
VERSION="20.04 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### GPU 資訊

GPU 的詳細資訊可使用 `!nvidia-smi` 指令。
如果沒有顯示畫面，可能是您沒有切換到 GPU 的環境喔!

In [18]:
!nvidia-smi

Thu Dec 12 14:24:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   33C    P8              15W / 230W |   1426MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from platform import python_version
import torch
print("Python version", python_version())
print("Pytorch - version", torch.__version__)
print("Pytorch - cuDNN version :", torch.backends.cudnn.version())

Python version 3.10.15
Pytorch - version 2.5.1+cu124
Pytorch - cuDNN version : 90100


### 資料上傳 (非常重要)
在 Google Colab 中，有以下幾種方式可以將資料上傳到虛擬機：

1. **從本機上傳 (From Machine)**
- 使用 Colab 左側內建的檔案管理面板。
- 點擊左側的檔案標籤，然後選擇 **上傳 (Upload)**，直接將本機的檔案拖放或選擇要上傳的檔案。

2. **從 Google Drive 上傳 (From Google Drive)**
- 在 Colab 左側內建的檔案管理面板中，點擊 **Mount Drive (掛載雲端硬碟)**。
- 掛載完成後，您的 Google Drive 會顯示在檔案樹中，您可以直接存取或載入檔案。

1. **使用 `wget` 指令**
- 如果資料存放在公開的網路上，您可以使用 `wget` 指令下載。
- 指令範例：
  ```bash
  !wget http://example.com/yourfile.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
!pwd

/home/is1ab/code2/CmdCaliper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
!wget https://dl.fbaipublicfiles.com/XLM/codes_xnli_100

--2024-12-12 14:31:50--  https://dl.fbaipublicfiles.com/XLM/codes_xnli_100
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.50, 13.35.7.38, 13.35.7.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.50|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 2973279 (2.8M) [text/plain]
Saving to: ‘codes_xnli_100’

codes_xnli_100      100%[===================>]   2.83M  4.97MB/s    in 0.6s    

2024-12-12 14:31:51 (4.97 MB/s) - ‘codes_xnli_100’ saved [2973279/2973279]



In [23]:
!head -n 10 codes_xnli_100

a n 58300410
e r 55874871
e n 48332651
i n 46367809
a r 42047014
s t 41963553
a l 34495577
e n</w> 33743928
o n 30604783
o r 30086868


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
!git clone https://github.com/ntunlp/MH6812-SPMS-2019

Cloning into 'MH6812-SPMS-2019'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 228 bytes | 228.00 KiB/s, done.


In [25]:
!ls -l CE-7455-SPMS-2019

ls: cannot access 'CE-7455-SPMS-2019': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


其他 Python 的操作請參閱 [Python CheetSheet](PythonCheatSheet_tw.ipynb)

### CmdCaliper

預設使用模型 [CyCraftAI/CmdCaliper](https://huggingface.co/CyCraftAI/CmdCaliper-small)，記得要先填寫相關同意書才能使用喔!


CmdCaliper 主要的兩隻程式 [synthesize_cmds.py](synthesize_cmds.py) 與 [synthesize_positive_cmds.py](synthesize_positive_cmds.py) 都至少需要一個 Inference Engine (例如 GPT)，這些模型的 API Key 可能需要付費才能使用，如果您沒有金鑰，這一部分可以不執行，直接看預覽結果～！

API Key 的取得請參考投影片或是對應模型的官網說明。

如果您有金鑰，請按照以下步驟。

1. ``%cp credential_config_template.yaml credential_config.yaml``
2. 編輯 [credential_config.yaml](credential_config.uaml) 填上您的金鑰


In [ ]:
%cp credential_config_template.yaml credential_config.yaml

In [ ]:
!pip3 install -r requirements.txt

In [2]:
from datasets import load_dataset 

ds = load_dataset("CyCraftAI/CyPHER") #取得資料集
ds

/home/is1ab/anaconda3/envs/cmdcaliper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['query_cmd', 'negative_index_list', 'positive_cmd', 'idx'],
        num_rows: 28520
    })
    test: Dataset({
        features: ['query_cmd', 'negative_index_list', 'positive_cmd', 'idx'],
        num_rows: 2807
    })
})

In [11]:
import pandas as pd
import json
import random

query_cmd_list = ds['train']['query_cmd']
random_query_cmd_list = random.sample(query_cmd_list, 20) # 隨機抽取 20 筆資料
random_query_cmd_list

['sigcheck -n C:\\Windows\\System32\\ntoskrnl.exe -nobanner',
 '\'wmic volume where "DriveType=3" set Label="Confidential Data"\'',
 'cscript "C:\\Windows\\System32\\Printing_Admin_Scripts\\en-US\\pubprn.vbs" "\\\\printserver\\HPLaserJet" ADD',
 'schtasks /create /tn MalwareTask /tr "C:\\Users\\User1\\Documents\\malware.exe" /sc DAILY /st 08:00',
 'powershell -Command "Get-Process | Select-Object -Property Name, ID, CPU"',
 'xcopy /s /e /i C:\\source\\* D:\\backup\\ (date:dd-MM-yyyy)',
 'C:\\WINDOWS\\system32\\rundll32.exe sysmain.dll,PfSvWsSwapAssessmentTask',
 'wmic os get FreePhysicalMemory /value | findstr "Data" > C:\\memory\\_report.txt',
 'sigcheck64.exe -accepteula -vt "C:\\Windows\\System32\\ntdll.dll"',
 'reg add "HKLM\\Software\\ExampleKey" /v ExampleValue /t REG_SZ /d "ExampleData" /f',
 'query.exe logonsession /list /username:company\\user28 /pw: mypassword /cachekms:1000 /v /port:9090',
 'msiexec /i "C:\\Users\\admin\\Downloads\\app_installer.msi" /qn /l*v "C:\\Users\\adm

In [12]:

import os

output_dir = "./dataset"
output_file = "query_cmd.json"

os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, output_file)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(random_query_cmd_list, f, ensure_ascii=False, indent=4)

In [ ]:
# Inference
# 這隻程式主要是依據輸入的 cmd 當作參考用來產生新的指令
# --max-generation-num 產生的 cmd 的總數! 所以一定要大於輸入的 cmd 數量
# 預設的 --fewshot-cmd-num = 12, 所以 query_cmd 數量要大於等於 12 或直接指定更大的數字

!python3 synthesize_cmds.py \
    --path-to-seed-cmds "./dataset/query_cmd.json" \
    --path-to-output-dir "./output" \
    --max-generation-num 40 \
    --path-to-credential-config ./credential_config.yaml


Initialize Engine - 'OpenAIInferenceEngine' Successfully!
Seed CMDs Num: 20
CMD Synthesis Progress: 20/40


In [14]:
# 第二隻程式 symthesize_positive_cmds 是產生相似的 cmdline
 
!python3 synthesize_positive_cmds.py \
    --path-to-all-cmds "./dataset/query_cmd.json" \
    --path-to-output-dir "./output" \
    --path-to-credential-config ./credential_config.yaml

Initialize Engine - 'OpenAIInferenceEngine' Successfully!
All Num of CMDs for Positive Synthesis: 20
Remaining Num of CMDs for Positive Synthesis: 20
Num of CMDs for Positive Synthesis: 20
Positive CMD Synthesis Progress: 0/20


### 評估程式 (核心貢獻)

In [44]:
!python3 evaluate.py --model-name CyCraftAI/CmdCaliper-small \
    --batch-size 16 --device cuda

Using the SentenceTransformerInferencer for the model - CyCraftAI/CmdCaliper-small
The performance of the model - CyCraftAI/CmdCaliper-small: ({3: 0.8681273007956477, 10: 0.8777671750329573}, {3: 0.8920555753473658, 10: 0.9476309226932882})


## Lab 1 - 根據敘述找到最接近的 cmdline

搜尋時測試大約會花費 1-2 分鐘，可以測試一個 Case 即可，資料庫約 6000 筆。

在一個安靜的辦公日，安全運維中心的監控系統突然報告多台伺服器上出現了異常活動。經過初步檢查，發現可能存在未經授權的權限提升行為，並且疑似攻擊者在伺服器中執行了一些命令，但具體內容無法完全掌握。

安全分析團隊決定根據現有的威脅情資描述，從 CyPHER 資料集中查找與攻擊行為相關的 cmdline，以便追蹤攻擊者的行為軌跡並評估影響範圍。

首席資安分析師發現駭客意圖：

「攻擊者利用 PowerShell 命令執行了一個惡意程式，並隱藏了執行過程的視窗。該命令使用 Start-Process 指令執行指定路徑下的惡意可執行檔案 exe.exe，同時以 -WindowStyle Hidden 隱藏視窗，降低使用者察覺的可能性。」

透過 GPT 請他幫忙產生對應的攻擊指令：
``powershell -Command "Start-Process 'C:\path\to\malicious\exe.exe' -PassThru" -WindowStyle Hidden``

In [19]:
!python3 cmd_retrieval.py --query-cmd 'powershell -Command "Start-Process 'C:\path\to\malicious\exe.exe' -PassThru" -WindowStyle Hidden' --model=CyCraftAI/CmdCaliper-small

Loading model: CyCraftAI/CmdCaliper-small
Querying for similar commands to: powershell -Command "Start-Process C:pathtomaliciousexe.exe -PassThru" -WindowStyle Hidden
Top similar commands:
Command: powershell -Command "Start-Process 'C:\path\to\malicious\exe.exe' -PassThru" -WindowStyle Hidden, Similarity Score: 0.8187
Command: powershell -NoProfile -ExecutionPolicy Bypass -WindowStyle Hidden -File C:\tmp\powershell-malicious-command, Similarity Score: 0.5743
Command: powershell.exe -ExecutionPolicy Bypass -Command "Set-MpPreference -DisableRealtimeMonitoring $true; Start-Process -FilePath 'C:\Program Files\Malware\malicious_app.exe' -WindowStyle Hidden; Set-MpPreference -DisableRealtimeMonitoring $false", Similarity Score: 0.5641
Command: powershell -ExecutionPolicy bypass -windowstyle hidden -file siphon.ps1, Similarity Score: 0.5627
Command: "powershell -windowstyle hidden -Command Start-Process powershell -Verb RunAs -ArgumentList "-noexit ; Get-Credential -Username fakeuser -Messa

一位安全分析師接到以下情資：

「攻擊者利用 Windows 任務排程工具 schtasks 創建了一個名為 "SystemUpdate" 的計劃任務。該任務設置為在午夜 00:00 時執行，並以最高的 SYSTEM 權限運行。該命令通過執行 cmd.exe 執行路徑中的惡意程式 C:\temp\malware.exe，以實現進一步的攻擊行為。」

透過 GPT 請他幫忙產生對應的攻擊指令：
``schtasks /create /tn "SystemUpdate" /tr "C:\Windows\System32\cmd.exe /c C:\temp\malware.exe" /sc once /st 00:00 /ru SYSTEM``

In [20]:
!python3 cmd_retrieval.py --query-cmd 'schtasks /create /tn "SystemUpdate" /tr "C:\Windows\System32\cmd.exe /c C:\temp\malware.exe" /sc once /st 00:00 /ru SYSTEM' --model=CyCraftAI/CmdCaliper-small

Loading model: CyCraftAI/CmdCaliper-small
Querying for similar commands to: schtasks /create /tn "SystemUpdate" /tr "C:\Windows\System32\cmd.exe /c C:\temp\malware.exe" /sc once /st 00:00 /ru SYSTEM
Top similar commands:
Command: schtasks /create /tn "MalwareUpdate" /tr "C:\malware_update.exe" /sc DAILY /st 00:00 /ru SYSTEM, Similarity Score: 0.8446
Command: schtasks /create /tn "MalwareUpdate" /tr "C:\malware\update.exe" /sc daily /st 12:00 /ru SYSTEM, Similarity Score: 0.8049
Command: schtasks /create /tn "MaliciousTask" /tr "C:\Windows\System32\malware.exe" /sc once /st 23:59 /ru system, Similarity Score: 0.7842
Command: schtasks /create /tn "Run Malware" /tr "C:\malware.exe" /sc once /st 00:00 /sd 01/01/2023 /ru "SYSTEM", Similarity Score: 0.7784
Command: "C:\Windows\System32\schtasks.exe" /CREATE /TN "Malware Execution" /TR "\"C:\Temp\malware.exe\"" /SC DAILY /ST 00:00, Similarity Score: 0.7698


後來一位安全分析師發現：

「攻擊者可能使用了一個命令來清除 Windows 系統的安全事件日誌，隱藏他們的足跡」

透過 GPT 請他幫忙產生對應的攻擊指令：
``wevtutil cl Security && wevtutil cl System && wevtutil cl Application && reg add "HKLM\SYSTEM\CurrentControlSet\Services\EventLog\Security" /v AutoBackupLogFiles /t REG_DWORD /d 0 /f && sc config EventLog start= disabled``

or 

``wevtutil cl Security``

In [21]:
!python3 cmd_retrieval.py --query-cmd 'wevtutil cl Security && wevtutil cl System && wevtutil cl Application && reg add "HKLM\SYSTEM\CurrentControlSet\Services\EventLog\Security" /v AutoBackupLogFiles /t REG_DWORD /d 0 /f && sc config EventLog start= disabled' --model=CyCraftAI/CmdCaliper-small

Loading model: CyCraftAI/CmdCaliper-small
Querying for similar commands to: wevtutil cl Security && wevtutil cl System && wevtutil cl Application && reg add "HKLM\SYSTEM\CurrentControlSet\Services\EventLog\Security" /v AutoBackupLogFiles /t REG_DWORD /d 0 /f && sc config EventLog start= disabled
Top similar commands:
Command: wevtutil cl SecurityLog && wevtutil cl System && wevtutil cl Application, Similarity Score: 0.5901
Command: reg  add "HKLM\System\CurrentControlSet\Services\SecurityHealthService" /v "Start" /t REG_DWORD /d "4" /f, Similarity Score: 0.5743
Command: wevtutil cl System & wevtutil cl Security & wevtutil cl Application & wevtutil cl Setup & wevtutil cl ForwardedEvents, Similarity Score: 0.5660
Command: wevtutil epl System C:\Logs\System.evtx && wevtutil epl Security C:\Logs\Security.evtx && wevtutil epl Application C:\Logs\Application.evtx, Similarity Score: 0.5641
Command: wevtutil cl Application && wevtutil cl Security && wevtutil cl System && wevtutil gl Applicatio

## Lab 2 - 根據給定的 cmdline，推論出三個可能的下一步指令

In [2]:
import json

json_data_list = []

with open('./dataset/countercraft_apt29-deceptions_results.json', 'r', encoding='utf-8') as file:
    json_data_list.append(json.load(file))

with open('./dataset/sentinelone-deceptions_apt29-deceptions_results.json', 'r', encoding='utf-8') as file:
    json_data_list.append(json.load(file))

def extract_criteria(data):
    criteria_list = []

    for adversary in data.get("Adversaries", []):
        for scenario, scenario_data in adversary.get("Deceptions_By_Step", {}).items():
            for step in scenario_data.get("Steps", []):
                for substep in step.get("Substeps", []):
                    criteria = substep.get("Criteria")
                    if criteria:
                        criteria_list.append(criteria)

    return criteria_list

cmd_sequence_list = []
for item in json_data_list:
    cmd_sequence_list.append(extract_criteria(item))
    
cmd_sequence_list


[['explorer.exe executes powershell.exe',
  'powershell.exe executes schemas ADS vis Get-Content and IEX',
  'powershell.exe executes Win32_BIOS via Get-WmiObject',
  'powershell.exe executes Win32_BIOS and Win32_ComputerSystem via Get-WmiObject',
  'powershell.exe executes Win32_PnPEntity via Get-WmiObject',
  'powershell.exe executes Win32_ComputerSystem via Get-WmiObject',
  'powershell.exe executes Win32_ComputerSystem via Get-WmiObject',
  'powershell.exe executes Win32_Process via Get-WmiObject',
  'powershell.exe queries for current path via Get-Item',
  'certutil.exe decodes kxwn.lock',
  'powershell.exe adds Webcache subkey via Net-ItemProperty',
  'powershell.exe executes powershell.exe',
  'powershell.exe connects to 192.168.0.4 over HTTPS',
  'powershell.exe connects to 192.168.0.4 over HTTPS',
  'powershell.exe enumerates c:\\windows\\system32 via Get-ChildItem',
  'powershell.exe modifies time attributes of kxwn.lock',
  'powershell.exe queries for AntiVirusProduct via Ge

In [75]:
import torch
import torch.nn.functional as F
from datasets import load_dataset, concatenate_datasets
from src.inferencer import SentenceTransformerInferencer

class CommandRetriever:
    DATASET_NAME = "CyCraftAI/CyPHER"
    def __init__(self, batch_size, device):
        raw_data = load_dataset(self.DATASET_NAME)
        self.data = concatenate_datasets([raw_data["test"], raw_data["train"]])
        self.batch_size = batch_size
        self.device = device

    def process_cmd_sequences(self, cmd_sequence_list, inferencer):
        cmd_list = []
        embedding_list = []
        for cmd_sequence in cmd_sequence_list:
            cmd_embeddings = self._batch_inference(cmd_sequence, inferencer)
            # Flatten embeddings to match query embedding shape
            flattened_embeddings = cmd_embeddings.view(-1, cmd_embeddings.size(-1))
            cmd_list.append(cmd_sequence)
            embedding_list.extend(flattened_embeddings.tolist())
        return {"cmd": cmd_list, "embeddings": embedding_list}

    def _batch_inference(self, sentence_list, inferencer):
        embeddings = []
        for i in range(0, len(sentence_list), self.batch_size):
            sub_sentences = sentence_list[i: min(i + self.batch_size, len(sentence_list))]
            sub_embeddings = inferencer(sub_sentences).to(self.device)
            embeddings.append(sub_embeddings)
        return torch.cat(embeddings, dim=0)

    @staticmethod
    def retrieve_similar_commands_from_cmd_sequences(query_cmd, processed_data, inferencer, device):
        with torch.no_grad():
            query_embedding = inferencer([query_cmd]).to(device)
            print(f"Query embedding shape: {query_embedding.shape}")  # Debug: Check query embedding shape

            dataset_cmds = processed_data["cmd"]
            dataset_embeddings = torch.tensor(processed_data["embeddings"]).to(device)
            print(f"Dataset embeddings shape: {dataset_embeddings.shape}")  # Debug: Check dataset embeddings shape

            cosine_similarities = F.cosine_similarity(query_embedding, dataset_embeddings, dim=-1)
            top_idx = cosine_similarities.argmax().item()

            # Retrieve the top similar command and its subsequent commands
            sequence_index = top_idx // len(dataset_cmds[0])  # Map flattened index back to original cmd index
            cmd_index = top_idx % len(dataset_cmds[0])  # Find position within the sequence

            command_sequence = dataset_cmds[sequence_index]
            subsequent_cmds = command_sequence[cmd_index:cmd_index + 3]  # Get up to 3 subsequent commands

            return sequence_index, subsequent_cmds



In [76]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inferencer = SentenceTransformerInferencer("CyCraftAI/CmdCaliper-small",device)
retriever = CommandRetriever(batch_size=16, device=device)

processed_data = retriever.process_cmd_sequences(cmd_sequence_list, inferencer)

In [77]:
processed_data

{'cmd': [['explorer.exe executes powershell.exe',
   'powershell.exe executes schemas ADS vis Get-Content and IEX',
   'powershell.exe executes Win32_BIOS via Get-WmiObject',
   'powershell.exe executes Win32_BIOS and Win32_ComputerSystem via Get-WmiObject',
   'powershell.exe executes Win32_PnPEntity via Get-WmiObject',
   'powershell.exe executes Win32_ComputerSystem via Get-WmiObject',
   'powershell.exe executes Win32_ComputerSystem via Get-WmiObject',
   'powershell.exe executes Win32_Process via Get-WmiObject',
   'powershell.exe queries for current path via Get-Item',
   'certutil.exe decodes kxwn.lock',
   'powershell.exe adds Webcache subkey via Net-ItemProperty',
   'powershell.exe executes powershell.exe',
   'powershell.exe connects to 192.168.0.4 over HTTPS',
   'powershell.exe connects to 192.168.0.4 over HTTPS',
   'powershell.exe enumerates c:\\windows\\system32 via Get-ChildItem',
   'powershell.exe modifies time attributes of kxwn.lock',
   'powershell.exe queries for

In [82]:
query_cmd = 'powershell.exe -Command "Get-ComputerInfo | Select-Object CsName, OSName"'
sequence_index, similar_cmds = CommandRetriever.retrieve_similar_commands_from_cmd_sequences(query_cmd, processed_data, inferencer, device)
print(f"Sequence index: {sequence_index}, Subsequent commands: {similar_cmds}")

Query embedding shape: torch.Size([1, 384])
Dataset embeddings shape: torch.Size([272, 384])
Sequence index: 0, Subsequent commands: ['powershell.exe enumerates computer name via GetComputerNameEx()', 'powershell.exe enumerates domain name via NetWkstaGetInfo()', 'powershell.exe enumerates current username via GetUserNameEx()']
